# Analyzing RCT data with Precision Adjustment

## Data

In this lab, we analyze the Pennsylvania re-employment bonus experiment, which was previously studied in "Sequential testing of duration data: the case of the Pennsylvania ‘reemployment bonus’ experiment" (Bilias, 2000), among others. These experiments were conducted in the 1980s by the U.S. Department of Labor to test the incentive effects of alternative compensation schemes for unemployment insurance (UI). In these experiments, UI claimants were randomly assigned either to a control group or one of five treatment groups. Actually, there are six treatment groups in the experiments. Here we focus on treatment group 4, but feel free to explore other treatment groups. In the control group the current rules of the UI applied. Individuals in the treatment groups were offered a cash bonus if they found a job within some pre-specified period of time (qualification period), provided that the job was retained for a specified duration. The treatments differed in the level of the bonus, the length of the qualification period, and whether the bonus was declining over time in the qualification period; see http://qed.econ.queensu.ca/jae/2000-v15.6/bilias/readme.b.txt for further details on data. 
  

In [1]:
## loading the data
Penn <- as.data.frame(read.table("../data/penn_jae.dat", header=T ))
n <- dim(Penn)[1]
p_1 <- dim(Penn)[2]
Penn<- subset(Penn, tg==4 | tg==0)
attach(Penn)

In [2]:
T4<- (tg==4)
summary(T4)

   Mode   FALSE    TRUE 
logical    3354    1745 

In [ ]:
head(Penn)

### Model 
To evaluate the impact of the treatments on unemployment duration, we consider the linear regression model:

$$
Y =  D \beta_1 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W')' = 0,
$$

where $Y$ is  the  log of duration of unemployment, $D$ is a treatment  indicators,  and $W$ is a set of controls including age group dummies, gender, race, number of dependents, quarter of the experiment, location within the state, existence of recall expectations, and type of occupation.   Here $\beta_1$ is the ATE, if the RCT assumptions hold rigorously.


We also consider interactive regression model:

$$
Y =  D \alpha_1 + D W' \alpha_2 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W', DW')' = 0,
$$
where $W$'s are demeaned (apart from the intercept), so that $\alpha_1$ is the ATE, if the RCT assumptions hold rigorously.

Under RCT, the projection coefficient $\beta_1$ has
the interpretation of the causal effect of the treatment on
the average outcome. We thus refer to $\beta_1$ as the average
treatment effect (ATE). Note that the covariates, here are
independent of the treatment $D$, so we can identify $\beta_1$ by
just linear regression of $Y$ on $D$, without adding covariates.
However we do add covariates in an effort to improve the
precision of our estimates of the average treatment effect.

### Analysis

We consider 

*  classical 2-sample approach, no adjustment (CL)
*  classical linear regression adjustment (CRA)
*  interactive regression adjusment (IRA)

and carry out robust inference using the *estimatr* R packages. 

# Carry out covariate balance check

This is done using "lm_robust" command which unlike "lm" in the base command automatically does the correct Eicher-Huber-White standard errors, instead othe classical non-robus formula based on the homoscdedasticity command.

In [ ]:
m <- lm(T4~(female+black+othrace+factor(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2)
library(lmtest)
library(sandwich)
coeftest(m, vcov = vcovHC(m, type="HC1"))

In [186]:
# get same dataframe
X <- as.data.frame( model.matrix(m) )

no_col <- attributes(m$coefficients)$names[is.na(m$coefficients)]

X1  <- X[, -which(names( X ) %in% no_col ) ]

save( X1, file = "../data/m_reg.RData")

### Generating similar qr decomposition

#### Replicate how lm drop variables [as discussed here](https://stackoverflow.com/questions/35410769/remove-perfectly-multicollinear-variables-from-data-frame)

In [ ]:
# qr.X <- qr(X, tol=1e-3, LAPACK = FALSE)
# (rnkX <- qr.X$rank)  ## 4 (number of non-collinear columns)
# (keep <- qr.X$pivot[seq_len(rnkX)])
# X2 <- X[,keep]

In [ ]:
# colnames(X[,-keep]) == attributes(m$coefficients)$names[is.na(m$coefficients)]

# colnames(X[,-keep]) 

# attributes(m$coefficients)$names[is.na(m$coefficients)]

We see that that even though this is a randomized experiment, balance conditions are failed.

# Model Specification

In [203]:
# model specifications


# no adjustment (2-sample approach)
formula_cl <- log(inuidur1)~T4

# adding controls
formula_cra <- log(inuidur1)~T4+ (female+black+othrace+factor(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2
# Omitted dummies: q1, nondurable, muld


ols.cl_reg <- lm(formula_cl)
ols.cra_reg <- lm(formula_cra)


ols.cl = coeftest(ols.cl_reg, vcov = vcovHC(ols.cl_reg, type="HC1"))
ols.cra = coeftest(ols.cra_reg, vcov = vcovHC(ols.cra_reg, type="HC1"))

print(ols.cl)
print(ols.cra)




t test of coefficients:

             Estimate Std. Error  t value Pr(>|t|)    
(Intercept)  2.027585   0.017004 119.2431  < 2e-16 ***
T4          -0.085455   0.035856  -2.3833  0.01719 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


t test of coefficients:

                       Estimate Std. Error t value  Pr(>|t|)    
(Intercept)           2.6058121  0.3679364  7.0822 1.615e-12 ***
T4                   -0.0796801  0.0355909 -2.2388 0.0252143 *  
female               -0.1146093  0.3163904 -0.3622 0.7171880    
black                -0.8990265  0.2402463 -3.7421 0.0001845 ***
othrace              -2.6269216  0.4541551 -5.7842 7.729e-09 ***
factor(dep)1         -0.7196978  0.5812182 -1.2383 0.2156788    
factor(dep)2         -0.0405584  0.3598395 -0.1127 0.9102631    
q2                   -0.1596700  0.3704317 -0.4310 0.6664596    
q3                   -0.5398906  0.3700794 -1.4589 0.1446691    
q4                   -0.4333540  0.3711943 -1.1675 0.2430809    


In [191]:
# get same dataframe for ols.cl
X <- as.data.frame( model.matrix(ols.cl_reg) )

no_col <- attributes(ols.cl_reg$coefficients)$names[is.na(ols.cl_reg$coefficients)]

X1  <- X[, -which(names( X ) %in% no_col ) ]

save( X1, file = "../data/ols_cl_reg.RData")

# get same dataframe for ols.cl
X <- as.data.frame( model.matrix(ols.cra_reg) )

no_col <- attributes(ols.cra_reg$coefficients)$names[is.na(ols.cra_reg$coefficients)]

X1  <- X[, -which(names( X ) %in% no_col ) ]

save( X1, file = "../data/ols_cra_reg.RData")


The interactive specificaiton corresponds to the approach introduced in Lin (2013).

In [205]:

#interactive regression model;

X = model.matrix (~ (female+black+othrace+factor(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2)[,-1]
dim(X)
demean<- function(x){ x - mean(x)}
X = apply(X, 2, demean)

ols.ira_reg = lm(log(inuidur1) ~ T4*X) 
ols.ira= coeftest(ols.ira_reg, vcov = vcovHC(ols.ira_reg, type="HC1"))
print(ols.ira)




[1] 5099  119


t test of coefficients:

                            Estimate  Std. Error  t value  Pr(>|t|)    
(Intercept)               2.03177499  0.01687682 120.3885 < 2.2e-16 ***
T4                       -0.07550055  0.03560489  -2.1205 0.0340132 *  
Xfemale                  -0.12942734  0.31860760  -0.4062 0.6845928    
Xblack                   -0.92100431  0.24598526  -3.7441 0.0001831 ***
Xothrace                 -3.29328494  0.63054001  -5.2230 1.834e-07 ***
Xfactor(dep)1            -0.76246913  0.50426996  -1.5120 0.1305919    
Xfactor(dep)2             0.01931125  0.35130594   0.0550 0.9561647    
Xq2                      -0.13711567  0.39125789  -0.3504 0.7260173    
Xq3                      -0.52117189  0.39083482  -1.3335 0.1824351    
Xq4                      -0.41958694  0.39156262  -1.0716 0.2839658    
Xq5                      -0.33617506  0.38967611  -0.8627 0.3883426    
Xq6                      -0.47296422  0.39510320  -1.1971 0.2313392    
Xagelt35                 -0.61482757  

In [207]:
sum(S)

[1] 5276.375

In [206]:
# get same dataframe for ols.ira
S <- as.data.frame( model.matrix(ols.ira_reg) )

no_col <- attributes(ols.ira_reg$coefficients)$names[is.na(ols.ira_reg$coefficients)]

S1  <- S[, -which(names( S ) %in% no_col ) ]

save( S1, file = "../data/ols_ira_reg.RData")



Next we try out partialling out with lasso

In [198]:
library(hdm)

T4 = demean(T4)

DX = model.matrix(~T4*X)[,-1]

rlasso.ira = summary(rlassoEffects(DX, log(inuidur1), index = 1))


print(rlasso.ira)



[1] "Estimates and significance testing of the effect of target variables"
   Estimate. Std. Error t value Pr(>|t|)  
T4  -0.07889    0.03555  -2.219   0.0265 *
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1




In [201]:
# getting data
S = as.data.frame(DX)
save( S, file = "../data/rlasso_ira_reg.RData")

### Results

In [9]:
library(xtable)
table<- matrix(0, 2, 4)
table[1,1]<-  ols.cl[2,1]
table[1,2]<-  ols.cra[2,1]
table[1,3]<-  ols.ira[2,1]
table[1,4]<-  rlasso.ira[[1]][1]

table[2,1]<-  ols.cl[2,2]
table[2,2]<-  ols.cra[2,2]
table[2,3]<-  ols.ira[2,2]
table[2,4]<-  rlasso.ira[[1]][2]


colnames(table)<- c("CL","CRA","IRA", "IRA w Lasso")
rownames(table)<- c("estimate", "standard error")
tab<- xtable(table, digits=5)
tab

print(tab, type="latex", digits=5)

,CL,CRA,IRA,IRA w Lasso
,<dbl>,<dbl>,<dbl>,<dbl>
estimate,-0.08545541,-0.07968012,-0.07550055,-0.07888608
standard error,0.03585569,0.03559092,0.03560489,0.03555130


% latex table generated in R 3.6.3 by xtable 1.8-4 package
% Sat Feb 20 13:42:45 2021
\begin{table}[ht]
\centering
\begin{tabular}{rrrrr}
  \hline
 & CL & CRA & IRA & IRA w Lasso \\ 
  \hline
estimate & -0.08546 & -0.07968 & -0.07550 & -0.07889 \\ 
  standard error & 0.03586 & 0.03559 & 0.03560 & 0.03555 \\ 
   \hline
\end{tabular}
\end{table}


Treatment group 4 experiences an average decrease of about $7.8\%$ in the length of unemployment spell.


Observe that regression estimators delivers estimates that are slighly more efficient (lower standard errors) than the simple 2 mean estimator, but essentially all methods have very similar standard errors. From IRA results we also see that there is not any statistically detectable heterogeneity.  We also see the regression estimators offer slightly lower estimates -- these difference occur perhaps to due minor imbalance in the treatment allocation, which the regression estimators try to correct.


